In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
#import our data from balanced_filled_Dataset-vf.csv as data
data = pd.read_csv('balanced_filled_Dataset-vf.csv')
#split the data as 80% training and 20% testing
X = data.iloc[:,0:14]
Y = data.iloc[:,14]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state = 0, test_size = 0.2)
#scale the data
scale_X = StandardScaler()
X_train = scale_X.fit_transform(X_train)
X_test = scale_X.fit_transform(X_test)
#use KNN as our classifier - I used k = 39 because sqrt of length of y_test = 39 and this is a common way to guess the value of k, and it is odd number
classifier = KNeighborsClassifier(n_neighbors = 1, p = 7, metric = 'euclidean')
classifier.fit(X_train,Y_train)
#calculate the error
error = 1 - classifier.score(X_test,Y_test)
print("Error rate = " , 100*error, "%")

Error rate =  13.697749196141483 %
